In [1]:
! pip install folium
import folium as folium
import pandas as pd
import numpy as np
from branca.colormap import linear


CENTER_US = (39.8333333,-98.585522)
state_geojson = 'https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json'
map = folium.Map(location=CENTER_US, zoom_start=4)

ev_stations = pd.read_csv('https://raw.githubusercontent.com/jhumphries5/IST256-Group49/master/EVChargingStations_US_2020_04_19a.csv')

State_stations = ev_stations.groupby(['State'],as_index=False)
State_stations = State_stations.sum().filter(["State","Count"])

States_LatLong = pd.read_csv('https://raw.githubusercontent.com/jhumphries5/IST256-Group49/master/IST256-Project-StatesData.csv')

State_population = pd.read_csv('https://raw.githubusercontent.com/jhumphries5/IST256-Group49/master/populationdata2019-ist256C.csv')

df_merged1 = pd.merge(State_stations,States_LatLong)

df_merged2 = pd.merge(df_merged1,State_population)

df_merged2['TotalStationRank'] = df_merged2['Count'].rank(ascending = 0) 

df_merged2['StationDensity'] = np.float
df_merged2['StationDensity'] = 100 * (df_merged2["Count"] / df_merged2["SQMiles"])
df_merged2['DensityRank'] = df_merged2['StationDensity'].rank(ascending = 0) 
df_merged2['PopRank'] = np.float
df_merged2['PopRank'] = df_merged2['Population'].rank(ascending = 0)
df_merged2['StationsPerCapita'] = np.float
df_merged2['StationsPerCapita'] = 10000 * (df_merged2["Count"] / df_merged2["Population"])
df_merged2['PerCapitaRank'] = df_merged2['StationsPerCapita'].rank(ascending = 0)
df_merged2


title_html='''<h3 align="center" style="font-size:20px"><b>Electric Charging Stations in America</b></h3>'''
map.get_root().html.add_child(folium.Element(title_html))


feature_group0 = folium.FeatureGroup(name='Total Stations',overlay=False,show=True).add_to(map)
feature_group1 = folium.FeatureGroup(name='Density',overlay=False).add_to(map)
feature_group2 = folium.FeatureGroup(name='Per Capita',overlay=False).add_to(map)


choropleth0 = folium.Choropleth(
    geo_data=state_geojson,
    data=df_merged2,
    columns=['StateName','Count','TotalStationRank'],
    key_on ='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name= 'TotalStations',
    highlight=True,
    line_color='black',
    colormap = linear.YlGn_09.scale(0,6500).to_step(10)
).geojson.add_to(feature_group0)


choropleth1 = folium.Choropleth(
    geo_data=state_geojson,
    data=df_merged2,
    columns=['StateName','StationDensity','DensityRank'],
    key_on ='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name= 'Density',
    highlight=True,
    line_color='black',
    colormap = linear.YlGn_09.scale(0,20).to_step(10)
).geojson.add_to(feature_group1)

choropleth2 = folium.Choropleth(
    geo_data=state_geojson,
    data=df_merged2,
    columns=['StateName','StationsPerCapita','PerCapitaRank'],
    key_on ='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name= 'PerCapita',
    highlight=True,
    line_color='black',
    colormap = linear.YlGn_09.scale(0,5).to_step(10)
).geojson.add_to(feature_group2)

folium.LayerControl(collapsed=False).add_to(map)

for row in df_merged2.to_records():
    folium.Marker(
            location=(row['Latitude'],row['Longitude']), 
            popup=str(row['StateName'])+", Total Stations: "+str(row['Count']) +", Rank: "+str(row['TotalStationRank'])
                          ).add_to(feature_group0)
    
for row in df_merged2.to_records():
    folium.Marker(
            location=(row['Latitude'],row['Longitude']), 
            popup=str(row['StateName'])+", Stations Per 100 Miles: "+str(row['StationDensity']) +", Rank: "+str(row['DensityRank'])
                          ).add_to(feature_group1)
    
for row in df_merged2.to_records():
    folium.Marker(
            location=(row['Latitude'],row['Longitude']), 
            popup=str(row['StateName'])+", Stations Per 1000 People: "+str(row['StationsPerCapita']) +", Rank: "+str(row['PerCapitaRank'])
                          ).add_to(feature_group2)
    
map.save('Electric_Charging_Stations.html')

map
